<a href="https://colab.research.google.com/github/ejshumaker/EECS504Project/blob/master/EECS504FinalProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#EECS 504 Final Project

### Bryan Edwards, Demba Komma, Leah Wu, Eric Shumaker
#### {bryaned, dbkomma, huiminwu, eshumake} @umich.edu


# Starting


In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torchvision
from torchvision import datasets, models, transforms
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F

print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)
# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    print("Using the GPU!")
else:
    print("WARNING: Could not find GPU! Using CPU only. If you want to enable GPU, please to go Edit > Notebook Settings > Hardware Accelerator and select GPU.")


PyTorch Version:  1.4.0
Torchvision Version:  0.5.0
Using the GPU!


## [1] Downloading the dataset.



In [0]:
# TODO

## [2] Build dataloaders for train and test

In [0]:
# TODO

## [3] Define Architecture

In [0]:
# TODO

## [4] Train



In [0]:
# TODO

## [5] Visualization



In [0]:
# TODO